# Notebook to demonstrate visualizing external metabolite exchanges and reaction fluxes between three organisms

# toc

## Load modules

In [1]:
# Move to parent directory for CONCERTO
import os
os.chdir('..')


In [2]:
import cobra
import json
import math
import pandas as pd
import numpy as np
import d3flux as d3f
from viz_utils import *

import viz_utils

import importlib


## Load Organism Models

In [3]:
from models import syn, av, rt

# av = cobra.io.read_sbml_model('../models/azotobacter_vinelandii_dj/iAA1300.xml')#cobra.io.read_sbml_model('../models/azotobacter_vinelandii_dj/iDT1278.xml')
# syn = cobra.io.read_sbml_model('../models/synechococcus_elongatus_pcc_7942/iJB785.xml')
# rt = cobra.io.read_sbml_model('../models/rhodosporidium_toruloides_ifo_08804/Rt_IFO0880.xml')

In [4]:
av

Name,AV
Memory address,7f9b85ce8760
Number of metabolites,2003
Number of reactions,2469
Number of genes,1278
Number of groups,60
Objective expression,1.0*BIOMASS_Av_DJ_core - 1.0*BIOMASS_Av_DJ_core_reverse_428bc
Compartments,"c, e, p"


### Update model ids: 
The "Id_Id" format prevents erasing org name entirely for each organism node in visualization

In [5]:
# format ==> "Id_Id"

syn.id = "Se_Se"
rt.id = "Rt_Rt"
av.id = "Av_Av"

## Demonstrate default d3flux visualization for combined model

### Create combined model for three organisms

In [6]:
# Create a model name used to label the new cobra model and save corresponding json files
combined_model_name = "AvRtSe_Metabolite_Exchange_Model"

av_syn_rt_exchange_model = viz_utils.make_met_exchange_model_only_shared_external_mets(
    [av, syn, rt],
    model_name = combined_model_name
)


In [7]:
av_syn_rt_exchange_model

Name,AvRtSe_Metabolite_Exchange_Model
Memory address,7f9b8f9ae560
Number of metabolites,155
Number of reactions,327
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,"Av_Av, e, Se_Se, Rt_Rt"


### Create classic d3flux map using the combined model created above

In [8]:
# Demonstrate default d3flux visualization with combined model
d3f.flux_map(av_syn_rt_exchange_model)

### Create organized d3flux map using the full combined model created above

In [10]:

# Define figure boundaries and spacing between nodes for x & y directions
x_vals = {
    'min': -90,
    'max': 830,
    'space': 135
}

y_vals = {
    'min': -20,
    'max': 720,
    'space': 20
}


fluxmap_dict = viz_utils.make_3organism_extmetab_viz(
    [av, syn, rt],
    model_name = combined_model_name,
    
    x_min = x_vals['min'],
    x_max = x_vals['max'],
    d_x = x_vals['space'],
    
    y_min = y_vals['min'],
    y_max = y_vals['max'],
    d_y = y_vals['space'],
)


display(fluxmap_dict["map"])

NOTE: Adjusting SHORTEST list center position away from originally defined location
Showing map corresponding to the following file: AvRtSe_Metabolite_Exchange_Model_Updated_5.json


### Create an organized d3flux map of the full combined model with fluxes

In [12]:
# Read in and manipulate df for flux info
flux_fname = "Unedited_Medium_Fluxes.csv"
flux_df = pd.read_csv(flux_fname)
display(flux_df.head())

# Create dict to match organism id's to df index/rownames 
orgname_col = 'compartment'
org_comp2id_dict = {
    flux_df[orgname_col][0]: av.id, #Azotobacter
    flux_df[orgname_col][1]: rt.id, #Rhodosporidium
    flux_df[orgname_col][2]: syn.id, #Synechococcus
    flux_df[orgname_col][3]: flux_df['compartment'][3] # medium, currently not used but needed to match nrow(df)
}

# Define the organism IDs as the df index
flux_df.index = [org_comp2id_dict[k] for k in org_comp2id_dict]

display(flux_df.head())


,compartment,10FTHFGLULL,10FTHFGLULLm,12DGR120tipp,12DGR140tipp,12DGR141tipp,12DGR160tipp,12DGR161tipp,12DGR180tipp,12DGR181tipp,...,ZXANHX,ZYMSTATer_RT,ZYMSTESTH_RT,ZYMSTESTtrd,ZYMSTt,ZYMSTtr,Zn2t,Zn2tex,ilvg,r0647
0,Azotobacter,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001399,0.0,NaN
1,Rhodosporidium,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.003397,0.0,0.000034,-0.269638,-0.269638,0.001019,NaN,NaN,0.0
2,Synechococcus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.008967,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
3,medium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,compartment,10FTHFGLULL,10FTHFGLULLm,12DGR120tipp,12DGR140tipp,12DGR141tipp,12DGR160tipp,12DGR161tipp,12DGR180tipp,12DGR181tipp,...,ZXANHX,ZYMSTATer_RT,ZYMSTESTH_RT,ZYMSTESTtrd,ZYMSTt,ZYMSTtr,Zn2t,Zn2tex,ilvg,r0647
Av_Av,Azotobacter,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001399,0.0,NaN
Rt_Rt,Rhodosporidium,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.003397,0.0,0.000034,-0.269638,-0.269638,0.001019,NaN,NaN,0.0
Se_Se,Synechococcus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.008967,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
medium,medium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# # Create dict to match organism id's to df index/rownames 
# org_name2row_dict = {
#     av.id: flux_df.index[0],
#     rt.id: flux_df.index[1],
#     syn.id: flux_df.index[2],
# }

In [14]:
# Identify reactions and metabolites corresponding to 0-valued fluxes while building a flux dict for d3flux viz
# Note: automatically exclude NaN entries

# Get model in json format
with open(fluxmap_dict["fname"], "r") as read_file:
    ex_met_exchange_map = json.load(read_file)

# User-define values
exclude_0flux = False
flux_threshold = None

excluded_reactions = []
excluded_metabolites = []
this_flux_dict = {}

for r in ex_met_exchange_map['reactions']:
    r_m = r['id'][:-6]
    org_id = r['id'][-5:]
    this_flux = flux_df.loc[org_id, flux_df.columns[flux_df.columns == r_m]].values[0]
    
    # Check conditions and combine them
    exclude_NaN_cond = np.isnan(this_flux)
    exclude_0flux_cond = this_flux==0 if exclude_0flux else False
    exclude_by_thresh_cond = (abs(this_flux)<flux_threshold) if (flux_threshold is not None) else False
    exclude_flux_cond = exclude_NaN_cond or exclude_0flux_cond or exclude_by_thresh_cond    
    
    if r_m.startswith("EX_no2"):
        print(r['id'])
        print(exclude_NaN_cond)
        print(exclude_0flux_cond)
        print(exclude_by_thresh_cond)
        print(exclude_flux_cond)
        
    # Collect info on fluxes and excluded reactions/metabolites
    if exclude_flux_cond:
        excluded_reactions.append(r['id'])
        #excluded_metabolites.append(r_m[3:])
        if not exclude_NaN_cond:
            this_flux_dict[r['id']]=0
    else:
        this_flux_dict[r['id']]=this_flux


new_dict = this_flux_dict

print(excluded_reactions)
display(this_flux_dict)

EX_no2_e_Av_Av
False
False
False
False
EX_no2_e_Se_Se
True
False
False
True
EX_no2_e_Rt_Rt
False
False
False
False
['EX_cysi__L_e_Av_Av', 'EX_tre_e_Av_Av', 'EX_mal__L_e_Av_Av', 'EX_rbt_e_Av_Av', 'EX_his__L_e_Se_Se', 'EX_no2_e_Se_Se']


{'EX_ala__L_e_Av_Av': 0.0,
 'EX_co2_e_Av_Av': 150.9467711,
 'EX_akg_e_Av_Av': 0.0,
 'EX_gsn_e_Av_Av': 0.0,
 'EX_btn_e_Av_Av': 0.0,
 'EX_ocdcea_e_Av_Av': 0.0,
 'EX_no2_e_Av_Av': 5.28,
 'EX_g6p_e_Av_Av': 0.0,
 'EX_ade_e_Av_Av': 0.000923109,
 'EX_cu2_e_Av_Av': -0.002908819,
 'EX_gln__L_e_Av_Av': -5.874422943,
 'EX_ca2_e_Av_Av': -0.02135459,
 'EX_g1p_e_Av_Av': 0.0,
 'EX_g3pe_e_Av_Av': 0.0,
 'EX_alltn_e_Av_Av': 0.0,
 'EX_f6p_e_Av_Av': -1.42e-14,
 'EX_ocdca_e_Av_Av': 0.0,
 'EX_but_e_Av_Av': 0.0,
 'EX_cobalt2_e_Av_Av': -0.000102568,
 'EX_metsox_R__L_e_Av_Av': -1.42e-14,
 'EX_zn2_e_Av_Av': -0.001399023,
 'EX_ura_e_Av_Av': 0.0,
 'EX_lys__L_e_Av_Av': -0.574809831,
 'EX_cellb_e_Av_Av': 0.0,
 'EX_sbt__D_e_Av_Av': 0.0,
 'EX_spmd_e_Av_Av': 0.0,
 'EX_thymd_e_Av_Av': 0.0,
 'EX_gmp_e_Av_Av': 0.0,
 'EX_gam6p_e_Av_Av': 0.0,
 'EX_nac_e_Av_Av': 0.0,
 'EX_5dglcn_e_Av_Av': 0.0,
 'EX_malt_e_Av_Av': 0.0,
 'EX_mn2_e_Av_Av': -0.002834971,
 'EX_lyx__L_e_Av_Av': 0.0,
 'EX_fru_e_Av_Av': 0.0,
 'EX_pyr_e_Av_Av': 79.7

In [15]:
# Select to exclude metabolites only if they don't have at least two reactions excluded 
for r_id in excluded_reactions:
    m_id = r_id[3:-6]
    all_r_same_m = [r['id'] for r in ex_met_exchange_map['reactions'] if r['id'].startswith(r_id[:-6])]
    excl_r_same_m = [r for r in excluded_reactions if r.startswith(r_id[:-6])]
    if (len(all_r_same_m)-len(excl_r_same_m))<2:
        excluded_metabolites.append(m_id)

print(excluded_metabolites)

['cysi__L_e', 'tre_e', 'mal__L_e', 'rbt_e']


In [16]:
print(type(fluxmap_dict["model"]))

for r in fluxmap_dict["model"].reactions:
    display(r.id)

# for r in fluxmap_dict["model"].reactions:
#     print(r['id'])

<class 'cobra.core.model.Model'>


'EX_ala__L_e_Av_Av'

'EX_cysi__L_e_Av_Av'

'EX_co2_e_Av_Av'

'EX_akg_e_Av_Av'

...

'EX_meoh_e_Rt_Rt'

'EX_rbt_e_Rt_Rt'

'EX_arbt_e_Rt_Rt'

'EX_btn_e_Rt_Rt'

'EX_pydxn_e_Rt_Rt'

In [17]:
# Remove excluded reactions and metabolites from the model
import copy

display(fluxmap_dict["model"].__dir__())

temp_model = copy.copy(fluxmap_dict["model"])

for r in excluded_reactions:
    model_r = temp_model.reactions.get_by_id(r)
    print(model_r)
    temp_model.remove_reactions([model_r])

for m in temp_model.metabolites:
    if m.id.startswith("no2"):
        print(m)

        #print(temp_model.metabolites.get_by_id(["no2_e"]))
        

['_id',
 'name',
 'notes',
 '_annotation',
 'genes',
 'reactions',
 'metabolites',
 'groups',
 '_compartments',
 '_contexts',
 '_solver',
 '_tolerance',
 '__module__',
 '__doc__',
 '__init__',
 '__setstate__',
 '__getstate__',
 'solver',
 'tolerance',
 'compartments',
 'medium',
 'copy',
 'add_metabolites',
 'remove_metabolites',
 'add_boundary',
 'add_reactions',
 'remove_reactions',
 'add_groups',
 'remove_groups',
 'get_associated_groups',
 'add_cons_vars',
 'remove_cons_vars',
 'problem',
 'variables',
 'constraints',
 'boundary',
 'exchanges',
 'demands',
 'sinks',
 '_populate_solver',
 'slim_optimize',
 'optimize',
 'repair',
 'objective',
 'objective_direction',
 'summary',
 '__enter__',
 '__exit__',
 'merge',
 '_repr_html_',
 'id',
 '_set_id_with_model',
 'annotation',
 '__repr__',
 '__str__',
 '__dict__',
 '__weakref__',
 '__new__',
 '__hash__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__reduce_ex_

EX_cysi__L_e_Av_Av: Av_Av --> cysi__L_e
EX_tre_e_Av_Av: Av_Av --> tre_e
EX_mal__L_e_Av_Av: Av_Av --> mal__L_e
EX_rbt_e_Av_Av: Av_Av --> rbt_e
EX_his__L_e_Se_Se: Se_Se --> his__L_e
EX_no2_e_Se_Se: Se_Se --> no2_e
no2_e


In [18]:
# Create vizualization with filtering by flux after one instance of reorganizing the node locations
# NOTE: this used d3flux directly along with the reorganized model layout from above
d3f.flux_map(fluxmap_dict["model"], #ex_met_exchange_model_new, 
#             excluded_reactions=excluded_reactions, 
#             excluded_metabolites=excluded_metabolites,
             flux_dict=this_flux_dict
            )

In [20]:
import viz_utils
importlib.reload(viz_utils)

# Define figure boundaries and spacing between nodes for x & y directions
x_vals = {'min': -90,
          'max': 830,
          'space': 135
           }
y_vals = {'min': -20,
          'max': 720,
          'space': 20
           }


fluxmap_dict, d2 = viz_utils.make_3organism_extmetab_viz([av, syn, rt],
                                                     model_name = combined_model_name,
                                                     flux_df=flux_df,
                                                     reorg_after_filter=False,
                                                     x_min = x_vals['min'], x_max = x_vals['max'], d_x = x_vals['space'],
                                                     y_min = y_vals['min'], y_max = y_vals['max'], d_y = y_vals['space'],
                                     )

# display(flux_dict_info['flux_dict'])
display(fluxmap_dict['map'])

NOTE: Adjusting SHORTEST list center position away from originally defined location
Showing map corresponding to the following file: AvRtSe_Metabolite_Exchange_Model_Updated_7.json


In [23]:
fluxmap_dict["map"] = d3f.flux_map(fluxmap_dict['model'],
                                   excluded_reactions=d2['excluded_reacts'],
                                   excluded_metabolites=d2['excluded_met'],
                                   flux_dict=d2['flux_dict']
                                  )
display(fluxmap_dict['map'])

In [22]:
import viz_utils
importlib.reload(viz_utils)


# Define figure boundaries and spacing between nodes for x & y directions
x_vals = {'min': -90,
          'max': 830,
          'space': 135
           }
y_vals = {'min': -20,
          'max': 720,
          'space': 20
           }

org_list = [av, syn, rt]
model_name = combined_model_name
flux_df=flux_df
reorg_after_filter=False
exclude_0flux=True

multi_org_model=None
reorg_after_filter=False
flux_threshold=None

x_min = x_vals['min']
x_max = x_vals['max']
d_x = x_vals['space']
y_min = y_vals['min']
y_max = y_vals['max']
d_y = y_vals['space']


# Create a multi-organism model if one isn't already provided
if multi_org_model == None:
    multi_org_model = viz_utils.make_met_exchange_model_only_shared_external_mets(org_list, model_name)

# Save multi-organism model in JSON format to work with
model_dict = viz_utils.save_default_d3fluxmap_to_json(multi_org_model)

shared_ext_m = viz_utils.ex_metab_reorg(org_list, flux_df, exclude_0flux, flux_threshold)



display(model_dict)
display(shared_ext_m)

NameError: name 'flux_df' is not defined

In [23]:
x_vals = {'min': -90,
          'max': 830,
          'space': 135
           }
y_vals = {'min': -20,
          'max': 720,
          'space': 20
           }
x_min = x_vals['min']
x_max = x_vals['max']
d_x = x_vals['space']
y_min = y_vals['min']
y_max = y_vals['max']
d_y = y_vals['space']



org_pos = {
    'longest_shortest': {'name': 'Rt_Rt', 'x': 216.66666666666663, 'y': 720}, 
    'longest_midlength': {'name': 'Av_Av', 'x': 523.3333333333333, 'y': -20}, 
    'shortest_midlength': {'name': 'Se_Se', 'x': 830, 'y': 473.3333333333333}
}



row_cols = {}

# Calculate about how many vertical space allocations go into a horizontal one
numR2Cwidth = int(d_x/d_y)

# The rest of this function will build a dict for the limits for each list of metabolites
# Each group of values will need to be determined sequentially

### Start with known limits
pos_limits = {
    'longest': {  # top-left corner
        'top_row_start': {  # top-left corner
            'x': x_min,
            'y': y_min
        },
        'bottom_row_end_max': {
            # one column left of longest_midlength organism; one row above of longest_shortest organism
            'x': org_pos['longest_midlength']['x'] - 0.5 * d_x,
            'y': org_pos['longest_shortest']['y'] - 0.5 * d_y
        }
    },
    'midlength': {  # right edge - top
        'top_row_end_min': {  # right side; one row below longest_midlength organism
            'x': x_max,
            'y': org_pos['longest_midlength']['y'] + d_y,
        },
        'bottom_row_end_max': {  # right side; one row above shortest_midlength organism
            'x': x_max,
            'y': org_pos['shortest_midlength']['y'] - d_y,
        },
        'right_col_center': {  # right edge-top
            'x': x_max,
            'y': y_min + (y_max - y_min) * (1.0 / 3.0)
        }
    },
    'shortest': {  # bottom edge - right
        'bottom_row_start_min': {  # bottom left side; one column right of longest_shortest organism
            'x': org_pos['longest_shortest']['x'] + d_x,
            'y': y_max - d_y
        },
        'bottom_row_end_max': {  # bottom right side; one column left of shortest_midlength organism
            'x': org_pos['shortest_midlength']['x'],
            'y': y_max
        },
        'bottom_row_center': {  # bottom_edge right
            'x': x_min + (x_max - x_min) * (2.0 / 3.0),
            'y': y_max - d_y
        }
    }
}



# Original
shared_ext_m = {'longest': {'2way_key': '1_3', 'orgs': [1, 3], 'org_names': ['Av_Av', 'Rt_Rt'], 'ext_m': ['12ppd__R_e', '12ppd__S_e', '2obut_e', '4abut_e', '5dglcn_e', 'abt__D_e', 'ac_e', 'acac_e', 'ade_e', 'adn_e', 'akg_e', 'ala_B_e', 'ala__D_e', 'ala__L_e', 'alltn_e', 'amp_e', 'arab__L_e', 'arbt_e', 'asn__L_e', 'asp__L_e', 'btn_e', 'but_e', 'bz_e', 'cellb_e', 'cgly_e', 'chol_e', 'cit_e', 'cmp_e', 'csn_e', 'cu_e', 'cysi__L_e', 'cytd_e', 'dad_2_e', 'dca_e', 'ddca_e', 'dha_e', 'din_e', 'dtmp_e', 'f6p_e', 'fmn_e', 'for_e', 'fru_e', 'fum_e', 'g1p_e', 'g3pc_e', 'g3pe_e', 'g3pi_e', 'g6p_e', 'gal_e', 'galt_e', 'galur_e', 'gam6p_e', 'glc__D_e', 'glcn_e', 'glx_e', 'glyc2p_e', 'glyc3p_e', 'glyc__R_e', 'glyc_e', 'glyclt_e', 'glycogen_e', 'gmp_e', 'gsn_e', 'gthrd_e', 'gua_e', 'h2o2_e', 'hdca_e', 'hdcea_e', 'hxa_e', 'inost_e', 'ins_e', 'lac__D_e', 'lac__L_e', 'lys__L_e', 'lyx__L_e', 'mal__L_e', 'malt_e', 'malttr_e', 'man6p_e', 'man_e', 'melib_e', 'metsox_R__L_e', 'metsox_S__L_e', 'minohp_e', 'nac_e', 'ocdca_e', 'ocdcea_e', 'orn_e', 'peamn_e', 'pnto__R_e', 'ppa_e', 'pser__L_e', 'pydxn_e', 'pyr_e', 'quin_e', 'rbt_e', 'rib__D_e', 'sbt__D_e', 'ser__D_e', 'ser__L_e', 'so3_e', 'srb__L_e', 'succ_e', 'sucr_e', 'taur_e', 'thm_e', 'thr__L_e', 'thym_e', 'thymd_e', 'tre_e', 'tsul_e', 'ttdca_e', 'ump_e', 'ura_e', 'urate_e', 'urea_e', 'uri_e', 'xan_e', 'xtsn_e', 'xyl__D_e', 'xylt_e']}, 'midlength': {'2way_key': '1_2', 'orgs': [1, 2], 'org_names': ['Av_Av', 'Se_Se'], 'ext_m': ['cobalt2_e', 'cynt_e', 'mobd_e', 'ni2_e']}, 'shortest': {'2way_key': '2_3', 'orgs': [2, 3], 'org_names': ['Se_Se', 'Rt_Rt'], 'ext_m': ['arg__L_e', 'hco3_e', 'leu__L_e', 'ptrc_e']}, 'threeway': {'orgs': [1, 2, 3], 'org_names': ['Av_Av', 'Se_Se', 'Rt_Rt'], 'ext_m': ['ca2_e', 'cl_e', 'co2_e', 'cu2_e', 'fe2_e', 'fe3_e', 'gln__L_e', 'h2o_e', 'h_e', 'his__L_e', 'k_e', 'meoh_e', 'mg2_e', 'mn2_e', 'na1_e', 'nh4_e', 'no2_e', 'no3_e', 'o2_e', 'pi_e', 'so4_e', 'spmd_e', 'zn2_e']}}

# Filtered
shared_ext_m = {'longest': {'2way_key': '1_3', 'orgs': [1, 3], 'org_names': ['Av_Av', 'Rt_Rt'], 'ext_m': ['12ppd__R_e', '12ppd__S_e', '2obut_e', '4abut_e', '5dglcn_e', 'abt__D_e', 'ac_e', 'acac_e', 'ade_e', 'adn_e', 'akg_e', 'ala_B_e', 'ala__D_e', 'ala__L_e', 'alltn_e', 'amp_e', 'arab__L_e', 'arbt_e', 'asn__L_e', 'asp__L_e', 'btn_e', 'but_e', 'bz_e', 'cellb_e', 'cgly_e', 'chol_e', 'cit_e', 'cmp_e', 'csn_e', 'cu_e', 'cytd_e', 'dad_2_e', 'dca_e', 'ddca_e', 'dha_e', 'din_e', 'dtmp_e', 'f6p_e', 'fmn_e', 'for_e', 'fru_e', 'fum_e', 'g1p_e', 'g3pc_e', 'g3pe_e', 'g3pi_e', 'g6p_e', 'gal_e', 'galt_e', 'galur_e', 'gam6p_e', 'glc__D_e', 'glcn_e', 'glx_e', 'glyc2p_e', 'glyc3p_e', 'glyc__R_e', 'glyc_e', 'glyclt_e', 'glycogen_e', 'gmp_e', 'gsn_e', 'gthrd_e', 'gua_e', 'h2o2_e', 'hdca_e', 'hdcea_e', 'his__L_e', 'hxa_e', 'inost_e', 'ins_e', 'lac__D_e', 'lac__L_e', 'lys__L_e', 'lyx__L_e', 'malt_e', 'malttr_e', 'man6p_e', 'man_e', 'melib_e', 'metsox_R__L_e', 'metsox_S__L_e', 'minohp_e', 'nac_e', 'no2_e', 'ocdca_e', 'ocdcea_e', 'orn_e', 'peamn_e', 'pnto__R_e', 'ppa_e', 'pser__L_e', 'pydxn_e', 'pyr_e', 'quin_e', 'rib__D_e', 'sbt__D_e', 'ser__D_e', 'ser__L_e', 'so3_e', 'srb__L_e', 'succ_e', 'sucr_e', 'taur_e', 'thm_e', 'thr__L_e', 'thym_e', 'thymd_e', 'tsul_e', 'ttdca_e', 'ump_e', 'ura_e', 'urate_e', 'urea_e', 'uri_e', 'xan_e', 'xtsn_e', 'xyl__D_e', 'xylt_e']}, 'midlength': {'2way_key': '1_2', 'orgs': [1, 2], 'org_names': ['Av_Av', 'Se_Se'], 'ext_m': ['cobalt2_e', 'cynt_e', 'mobd_e', 'ni2_e']}, 'shortest': {'2way_key': '2_3', 'orgs': [2, 3], 'org_names': ['Se_Se', 'Rt_Rt'], 'ext_m': ['arg__L_e', 'hco3_e', 'leu__L_e', 'ptrc_e']}, 'threeway': {'orgs': [1, 2, 3], 'org_names': ['Av_Av', 'Se_Se', 'Rt_Rt'], 'ext_m': ['ca2_e', 'cl_e', 'co2_e', 'cu2_e', 'fe2_e', 'fe3_e', 'gln__L_e', 'h2o_e', 'h_e', 'k_e', 'meoh_e', 'mg2_e', 'mn2_e', 'na1_e', 'nh4_e', 'no3_e', 'o2_e', 'pi_e', 'so4_e', 'spmd_e', 'zn2_e']}}




### Calculate values for the Longest two-way list: top left corner
L = len(shared_ext_m['longest']['ext_m'])




row_col_max = {
    'longest': {
        'r_max': int(
            (pos_limits['longest']['bottom_row_end_max']['y'] - pos_limits['longest']['top_row_start']['y']) / d_y),
        'c_max': int(
            (pos_limits['longest']['bottom_row_end_max']['x'] - pos_limits['longest']['top_row_start']['x']) / d_x)
    }
}

print(row_col_max)


ext_met_df = get_node_pos(org_pos, shared_ext_m, pos_limits, row_cols, d_x, d_y )

print(ext_met_df)

{'longest': {'r_max': 36, 'c_max': 4}}


KeyError: 'longest'